In [27]:
# auto update
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
#libs

# common
import pandas as pd
import numpy as np
import random as random

# language processing
import nltk
from nltk.stem.snowball import SnowballStemmer # lemmatization
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # vectorization
from nltk.corpus import stopwords
nltk.download('punkt_tab')

# mdata preparation
from sklearn.model_selection import train_test_split

# ML models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

# timer
import datetime

# metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# saving models
import pickle


[nltk_data] Downloading package punkt_tab to /home/andrey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [29]:
# language processing preparing
nltk.download("stopwords") # stop words removing
nltk.download('punkt') # split the text
nltk.download('wordnet') # lemmatization

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/andrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/andrey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
# load prepared models
model01 = pickle.load(open('models/model_01.mod', 'rb'))

In [31]:
# load special tables
x_BOW_table = pd.read_csv('data/x_BOW_names.csv', index_col=0, delimiter=',')
x_len = len(x_BOW_table)
x_line = [0] * x_len
x_line = pd.DataFrame(x_line).T
x_line.columns = x_BOW_table['names']
x_BOW_table = x_line
x_BOW_table.head() # do not like!

names,аа,ааа,аааа,ааааа,аааааааааааааааааааааааааааа,аалександр,аанасьев,аб,аба,абаев,...,ящериц,ящик,ящикдал,ящикещ,ящикздоров,ящичек,ящичк,ящук,ящуржинск,яявлен
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
data = 'Замечательный врач. Хочу выразить особую благодарность замечательному доктору - Замареновой Антонине Васильевне. Был сложный случай, сделала все отлично! И отдельная благодарность ее медсестре Оле, очень внимательная и вежливая девушка!'
data = pd.DataFrame([data], columns = ['text'])
data.head()

,text
0,Замечательный врач. Хочу выразить особую благо...


In [33]:
# clean the content -> only chars =======================================
for i in range(0, len(data)):
    data.loc[i, 'text'] = re.sub('[^а-яА-Я ]', '', data.loc[i, 'text'])
data.head()

,text
0,Замечательный врач Хочу выразить особую благод...


In [34]:
# tokenization -> split the text by tokens to list  =======================================
text_list = []
for i in range(0, len(data)):
    text_list.append(nltk.word_tokenize(data.loc[i, 'text'], language='russian'))
text_list[:1] # -> ok

[['Замечательный',
  'врач',
  'Хочу',
  'выразить',
  'особую',
  'благодарность',
  'замечательному',
  'доктору',
  'Замареновой',
  'Антонине',
  'Васильевне',
  'Был',
  'сложный',
  'случай',
  'сделала',
  'все',
  'отлично',
  'И',
  'отдельная',
  'благодарность',
  'ее',
  'медсестре',
  'Оле',
  'очень',
  'внимательная',
  'и',
  'вежливая',
  'девушка']]

In [35]:
# lemmatization -> to the base of simple word =======================================
Snow = SnowballStemmer('russian')

In [36]:
for i in range(0, len(text_list)):
    for j in range(0, len(text_list[i])):
        text_list[i][j] = Snow.stem(text_list[i][j])
text_list[:1] # -> ok        

[['замечательн',
  'врач',
  'хоч',
  'выраз',
  'особ',
  'благодарн',
  'замечательн',
  'доктор',
  'замаренов',
  'антонин',
  'васильевн',
  'был',
  'сложн',
  'случа',
  'сдела',
  'все',
  'отличн',
  'и',
  'отдельн',
  'благодарн',
  'е',
  'медсестр',
  'ол',
  'очен',
  'внимательн',
  'и',
  'вежлив',
  'девушк']]

In [37]:
# stop words removing =======================================
stop_words_russian = stopwords.words('russian')

In [38]:
text_list_cleaned = []
for i in range(0, len(text_list)):
    a = []
    for j in range(0, len(text_list[i])):
        if text_list[i][j] not in stop_words_russian:
            a.append(text_list[i][j])
    text_list_cleaned.append(a)
text_list_cleaned[:1] # -> ok     

[['замечательн',
  'врач',
  'хоч',
  'выраз',
  'особ',
  'благодарн',
  'замечательн',
  'доктор',
  'замаренов',
  'антонин',
  'васильевн',
  'сложн',
  'случа',
  'сдела',
  'отличн',
  'отдельн',
  'благодарн',
  'е',
  'медсестр',
  'ол',
  'очен',
  'внимательн',
  'вежлив',
  'девушк']]

In [39]:
# single chars still in the set
# preparing the set of single chars
alp='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
alp = alp + alp.upper()
alp  = [i for i in alp]
alp


['а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ё',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ё',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ъ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я']

In [40]:
text_list_cleaned2 = []
for i in range(0, len(text_list)):
    a = []
    for j in range(0, len(text_list[i])):
        if text_list[i][j] not in alp:
            a.append(text_list[i][j])
    text_list_cleaned2.append(a)
text_list_cleaned2[:1] # -> ok  

[['замечательн',
  'врач',
  'хоч',
  'выраз',
  'особ',
  'благодарн',
  'замечательн',
  'доктор',
  'замаренов',
  'антонин',
  'васильевн',
  'был',
  'сложн',
  'случа',
  'сдела',
  'все',
  'отличн',
  'отдельн',
  'благодарн',
  'медсестр',
  'ол',
  'очен',
  'внимательн',
  'вежлив',
  'девушк']]

In [41]:
# vectorization =======================================
Vectorizer = CountVectorizer()

In [42]:
# connect separeted words to the string
text_list_cleaned3 = []
for i in range(0, len(text_list)):
    text_list_cleaned3.append(' '.join(text_list_cleaned2[i]))
text_list_cleaned3[:2] # -> ok 

['замечательн врач хоч выраз особ благодарн замечательн доктор замаренов антонин васильевн был сложн случа сдела все отличн отдельн благодарн медсестр ол очен внимательн вежлив девушк']

In [43]:
# vectorization itself ===============================================
matrix_count = Vectorizer.fit_transform(text_list_cleaned3)

print('Vocab:', Vectorizer.vocabulary_)
print('names:', Vectorizer.get_feature_names_out())
print('array:', matrix_count.toarray())

Vocab: {'замечательн': 12, 'врач': 6, 'хоч': 22, 'выраз': 8, 'особ': 15, 'благодарн': 1, 'доктор': 10, 'замаренов': 11, 'антонин': 0, 'васильевн': 3, 'был': 2, 'сложн': 20, 'случа': 21, 'сдела': 19, 'все': 7, 'отличн': 17, 'отдельн': 16, 'медсестр': 13, 'ол': 14, 'очен': 18, 'внимательн': 5, 'вежлив': 4, 'девушк': 9}
names: ['антонин' 'благодарн' 'был' 'васильевн' 'вежлив' 'внимательн' 'врач'
 'все' 'выраз' 'девушк' 'доктор' 'замаренов' 'замечательн' 'медсестр' 'ол'
 'особ' 'отдельн' 'отличн' 'очен' 'сдела' 'сложн' 'случа' 'хоч']
array: [[1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1]]


In [44]:
# keep TFI-DF ===============================================
tfi_vectorizer = TfidfVectorizer()
tfi_matrix = tfi_vectorizer.fit_transform(raw_documents=text_list_cleaned3)

In [45]:
x_BOW_table2 = x_BOW_table
new_x_BOW = pd.DataFrame(matrix_count.toarray(), columns=[*Vectorizer.get_feature_names_out()])
new_x_BOW.head()

,антонин,благодарн,был,васильевн,вежлив,внимательн,врач,все,выраз,девушк,...,медсестр,ол,особ,отдельн,отличн,очен,сдела,сложн,случа,хоч
0,1,2,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [46]:
#d1.loc[0, d2.columns] = d2.iloc[0, :]
x_BOW_table2.loc[0, new_x_BOW.columns] = new_x_BOW.iloc[0,  :]
x_BOW_table2

,аа,ааа,аааа,ааааа,аааааааааааааааааааааааааааа,аалександр,аанасьев,аб,аба,абаев,...,ящериц,ящик,ящикдал,ящикещ,ящикздоров,ящичек,ящичк,ящук,ящуржинск,яявлен
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
len(x_BOW_table2.columns)

108644

In [48]:
x_BOW_table2 = x_BOW_table2.loc[:, x_BOW_table.columns]
len(x_BOW_table2.columns)

108644

In [49]:
answer01 = model01.predict(x_BOW_table2)
answer01[0]

/home/andrey/Documents/env/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


np.int64(1)